#### 自动创建eval/data文件夹下的yaml文件

In [ ]:
import os
import yaml

# 数据定义
DomainDataset = {
    'Energy': ['CAISO_AL', 'NYISO_AL', 'Wind'],
    'Healthcare': ['Covid-19', 'illness'],
    'Environment': ['weather', 'AQShunyi', 'AQWan'],
    'Nature': ['CzeLan', 'ZafNoo'],
    'Traffic': ['M_DENSE', 'METR_LA', 'NYCTAXI_Inflow', 'NYCTAXI_Outflow'],
    'Finance': ['Exchange_rate', 'IBM_m', 'MSFT_w', 'NASDAQ_w'],
    'Web': ['Wike2000', 'BizITObs-Application', 'BizITObs-Service', 'BizITObs-L2C'],
    'CloudOps': ['Bitbrains-rnd100-cpuusage', 'Bitbrains-rnd100-memoryusage']
}

Seqlen = {
    'Energy': ['long', 'long', 'long'],
    'Healthcare': ['short', 'short'],
    'Environment': ['long', 'long', 'long'],
    'Nature': ['long', 'long'],
    'Traffic': ['long', 'long', 'short', 'short'],
    'Finance': ['long', 'short', 'short', 'short'],
    'Web': ['short', 'long', 'long', 'long'],
    'CloudOps': ['short', 'short']
}

test_offset = {
    'Energy': ['7026', '13685', '38938'],
    'Healthcare': ['1114', '773'],
    'Environment': ['42157', '28051', '28051'],
    'Nature': ['15947', '15380'],
    'Traffic': ['14016', '27418', '1747', '1747'],
    'Finance': ['6070', '606', '1627', '618'],
    'Web': ['634', '7067', '7068', '25574'],
    'CloudOps': ['1493', '1493']
}

# 输出目录
output_dir = "/home/xiongshiwei/uni2ts/cli/conf/eval/data/"
os.makedirs(output_dir, exist_ok=True)

# 生成 yaml 文件
for domain, datasets in DomainDataset.items():
    offsets = test_offset[domain]
    seqlens = Seqlen[domain]
    for i, dataset in enumerate(datasets):
        content = {
            "_target_": "uni2ts.eval_util.data.get_custom_eval_dataset",
            "dataset_name": f"{dataset}_eval",
            "offset": int(offsets[i]),
            "windows": "???",
            "distance": 1,
            "prediction_length": "???",
            "mode": None,
            "max_history": 512 if seqlens[i] == 'long' else 128,
        }

        file_path = os.path.join(output_dir, f"{dataset}_test.yaml")
        with open(file_path, 'w') as f:
            yaml.dump(content, f, default_flow_style=False, sort_keys=False)

print(f"✅ 所有 YAML 文件已生成于: {output_dir}")

✅ 所有 YAML 文件已生成于: /home/xiongshiwei/uni2ts/cli/conf/eval/data/


#### 自动创建eval/model文件夹下的yaml文件

In [ ]:
import os
import yaml
from glob import glob

# 输入数据
DomainDataset = {
    'Energy': ['CAISO_AL', 'NYISO_AL', 'Wind'],
    'Healthcare': ['Covid-19', 'illness'],
    'Environment': ['weather', 'AQShunyi', 'AQWan'],
    'Nature': ['CzeLan', 'ZafNoo'],
    'Traffic': ['M_DENSE', 'METR_LA', 'NYCTAXI_Inflow', 'NYCTAXI_Outflow'],
    'Finance': ['Exchange_rate', 'IBM_m', 'MSFT_w', 'NASDAQ_w'],
    'Web': ['Wike2000', 'BizITObs-Application', 'BizITObs-Service', 'BizITObs-L2C'],
    'CloudOps': ['Bitbrains-rnd100-cpuusage', 'Bitbrains-rnd100-memoryusage']
}

Seqlen = {
    'Energy': ['long', 'long', 'long'],
    'Healthcare': ['short', 'short'],
    'Environment': ['long', 'long', 'long'],
    'Nature': ['long', 'long'],
    'Traffic': ['long', 'long', 'short', 'short'],
    'Finance': ['long', 'short', 'short', 'short'],
    'Web': ['short', 'long', 'long', 'long'],
    'CloudOps': ['short', 'short']
}

# 输出目录
output_dir = "/home/xiongshiwei/uni2ts/cli/conf/eval/model/"
os.makedirs(output_dir, exist_ok=True)

# 检查点路径模板
# ckpt_base_path = "/home/xiongshiwei/uni2ts/outputs/finetune/moirai_1.0_R_small"
# ckpt_base_path = "/home/xiongshiwei/uni2ts/outputs/finetune/moirai_1.0_R_base"
ckpt_base_path = "/home/xiongshiwei/uni2ts/outputs/finetune/moirai_1.0_R_large"



# 生成 YAML 文件
for domain, datasets in DomainDataset.items():
    for i, dataset in enumerate(datasets):
        seqlen_type = Seqlen[domain][i]
        context_length = 512 if seqlen_type == "long" else 128

        ckpt_dir = os.path.join(ckpt_base_path, dataset, dataset, "checkpoints")
        ckpt_files = glob(os.path.join(ckpt_dir, "*.ckpt"))
        checkpoint_path = ckpt_files[0] if ckpt_files else "MISSING_CKPT_FILE"

        config = {
            "_target_": "uni2ts.model.moirai.MoiraiForecast.load_from_checkpoint",
            "checkpoint_path": checkpoint_path,
            "num_samples": 100,
            "patch_size": 32,
            "context_length": context_length
        }

        yaml_path = os.path.join(output_dir, f"{dataset}.yaml")
        with open(yaml_path, "w") as f:
            yaml.dump(config, f, sort_keys=False)

output_dir


'/home/xiongshiwei/uni2ts/cli/conf/eval/model/'

#### 自动生成eval.sh中的脚本内容

In [ ]:
test_length = {
    'Energy': ['1757', '3421', '9735'],
    'Healthcare': ['278', '193'],
    'Environment': ['10539', '7013', '7013'],
    'Nature': ['3987', '3845'],
    'Traffic': ['3504', '6854', '437', '437'],
    'Finance': ['1518', '152', '407', '155'],
    'Web': ['158', '1767', '1767', '6394'],
    'CloudOps': ['373', '373']
}

DomainDataset = {
    'Energy': ['CAISO_AL', 'NYISO_AL', 'Wind'],
    'Healthcare': ['Covid-19', 'illness'],
    'Environment': ['weather', 'AQShunyi', 'AQWan'],
    'Nature': ['CzeLan', 'ZafNoo'],
    'Traffic': ['M_DENSE', 'METR_LA', 'NYCTAXI_Inflow', 'NYCTAXI_Outflow'],
    'Finance': ['Exchange_rate', 'IBM_m', 'MSFT_w', 'NASDAQ_w'],
    'Web': ['Wike2000', 'BizITObs-Application', 'BizITObs-Service', 'BizITObs-L2C'],
    'CloudOps': ['Bitbrains-rnd100-cpuusage', 'Bitbrains-rnd100-memoryusage']
}

Seqlen = {
    'Energy': ['long', 'long', 'long'],
    'Healthcare': ['short', 'short'],
    'Environment': ['long', 'long', 'long'],
    'Nature': ['long', 'long'],
    'Traffic': ['long', 'long', 'short', 'short'],
    'Finance': ['long', 'short', 'short', 'short'],
    'Web': ['short', 'long', 'long', 'long'],
    'CloudOps': ['short', 'short']
}

# 输出 bash 脚本
bash_lines = []

for domain in DomainDataset:
    datasets = DomainDataset[domain]
    lengths = test_length[domain]
    seq_types = Seqlen[domain]
    
    for i, dataset in enumerate(datasets):
        test_len = lengths[i]
        seq_type = seq_types[i]
        
        pl_set = ["96", "192", "336"] if seq_type == "long" else ["24", "48", "60"]
        
        bash_lines.append(f"# ========== {dataset} ({seq_type}, test_length={test_len}) ==========")
        bash_lines.append(f"for pl in {' '.join(pl_set)}; do")
        bash_lines.append(f"  echo \"eval {dataset} with prediction length $pl\"")
        bash_lines.append(f"  python -m cli.eval \\")
        bash_lines.append(f"    run_name={dataset} \\")
        bash_lines.append(f"    model={dataset} \\")
        bash_lines.append(f"    data={dataset}_test \\")
        bash_lines.append(f"    data.windows=$(( {test_len} - pl )) \\")
        bash_lines.append(f"    data.prediction_length=$pl")
        bash_lines.append(f"done\n")

# 打印生成的 bash 脚本
print("\n".join(bash_lines))


# ========== CAISO_AL (long, test_length=1757) ==========
for pl in 96 192 336; do
  echo "eval CAISO_AL with prediction length $pl"
  python -m cli.eval \
    run_name=CAISO_AL \
    model=CAISO_AL \
    data=CAISO_AL_test \
    data.windows=$(( 1757 - pl )) \
    data.prediction_length=$pl
done

# ========== NYISO_AL (long, test_length=3421) ==========
for pl in 96 192 336; do
  echo "eval NYISO_AL with prediction length $pl"
  python -m cli.eval \
    run_name=NYISO_AL \
    model=NYISO_AL \
    data=NYISO_AL_test \
    data.windows=$(( 3421 - pl )) \
    data.prediction_length=$pl
done

# ========== Wind (long, test_length=9735) ==========
for pl in 96 192 336; do
  echo "eval Wind with prediction length $pl"
  python -m cli.eval \
    run_name=Wind \
    model=Wind \
    data=Wind_test \
    data.windows=$(( 9735 - pl )) \
    data.prediction_length=$pl
done

# ========== Covid-19 (short, test_length=278) ==========
for pl in 24 48 60; do
  echo "eval Covid-19 with prediction leng